# Run GriddedCCP

In [1]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [2]:
import neuralproc

In [3]:
import submitit
from train_imgs import main, parse_arguments

log_folder="logs/%j"

In [4]:
def repeat_list(l, n):
    return [i for i in l for _ in range(n)]

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train=True 
        return submitit.utils.DelayedSubmission(self, args)
    
    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [7]:
datasets = ["celeba32", "celeba64", "mnist", "zs-multi-mnist"]

args = [parse_arguments("GridedCCP celeba64 --starting-run 0 --is-continue-train".split())]

In [8]:
executor=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*5, 
                           #partition="dev", 
                           cpus_per_task=10, 
                           mem='32GB',  
                          )

In [11]:
job = executor.map_array(Run(), args)

In [25]:
j

SlurmJob<job_id=17773846, task_id=0, state="UNKNOWN">

In [18]:
j = job[0]
job

TypeError: 'SlurmJob' object is not subscriptable

In [37]:
for j in job:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2019-09-17 00:31:22,761) - Starting with JobEnvironment(job_id=17765568_0, hostname=learnfair0261, local_rank=0(3), node=0(1), global_rank=0(3))
submitit INFO (2019-09-17 00:31:22,761) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/17765568_0/17765568_0_submitted.pkl

--- Training celeba32/GridedCCP/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1       -2.8506       -3.1677     +  552.6910
      2       -3.0912       -3.0443        458.0152
      3       -3.1353       -3.1300        428.5510
      4       -3.1516       -3.1497        432.9612
      5       -3.1546       -3.2349     +  430.9093
      6       -3.1751       -3.1969        447.8045
      7       -3.1788       -2.9709        446.0412
      8       -3.1884       -3.1852        436.9370
      9       -3.1744       -3.2008        439.4301
     10       -3.1958       -3.1877      

In [38]:
for j in job:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

INFO:submitit:Job completed successfully

--------------------------------
slurmstepd-learnfair0482: error: *** STEP 17765609.0 ON learnfair0482 CANCELLED AT 2019-09-17T06:49:10 ***
submitit WARNING (2019-09-17 06:49:10,351) - Bypassing sigterm.
submitit WARNING (2019-09-17 06:57:08,290) - Caught USR1 signal 10 on learnfair0482: this job is preempted.
INFO:submitit:Calling checkpoint method.
INFO:submitit:Requeued job 17765568_1 (3 remaining timeouts)
INFO:submitit:Exiting gracefully after preemption/timeout.
slurmstepd-learnfair0482: error: *** JOB 17765609 ON learnfair0482 CANCELLED AT 2019-09-17T06:57:20 DUE TO JOB REQUEUE ***
submitit WARNING (2019-09-17 06:57:20,281) - Bypassing sigterm.
submitit ERROR (2019-09-17 07:04:07,463) - Submitted job triggered an exception
ERROR:submitit:Submitted job triggered an exception
Traceback (most recent call last):
  File "/priva